# 環境構築

Azure Machine Learningジョブとしてスクリプトを実行する場合、ジョブ実行の実行コンテキストを定義する必要があります。重要な設定の1つは、スクリプトを実行するコンピュートターゲットです。これは、ローカルワークステーション（この場合はコンピュートインスタンス）、またはオンデマンドでプロビジョニングされるAzure Machine Learningマネージドコンピュートクラスターのようなリモートコンピュートターゲットになります。

このノートブックでは、コンピュートクラスタを作成し、ジョブ用のコンピュートターゲットを探索します。

## 始める前に

このノートブックのコードを実行するには、**azure-ai-ml** パッケージの最新バージョンが必要です。以下のセルを実行して、インストールされていることを確認してください。


In [ ]:
pip show azure-ai-ml

## Connect to your workspace

必要な SDK パッケージがインストールされ、ワークスペースに接続する準備が整いました。

ワークスペースに接続するには、識別子のパラメータ（サブスクリプションID、リソースグループ名、ワークスペース名）が必要です。Azure Machine Learningが管理するコンピュートインスタンスで作業しているので、ワークスペースに接続するにはデフォルト値を使用できる。

In [1]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [2]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## Run a script as a job

モデルをトレーニングするには、まず **src** フォルダに **diabetes_training.py** スクリプトを作成します。このスクリプトは、学習データと同じフォルダにある **diabetes.csv** ファイルを使用します。

スクリプトの最初にライブラリをインポートしていることに注意してください。これらのライブラリの関数は、データの処理とモデルの学習に使用されます。スクリプトの実行に使用する計算機には、これらのライブラリがインストールされている必要があります。

In [4]:
%%writefile src/diabetes-training.py
# import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# load the diabetes dataset
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# separate features and labels
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# split data into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# set regularization hyperparameter
reg = 0.01

# train a logistic regression model
print('Training a logistic regression model with regularization rate of', reg)
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# calculate accuracy
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)

# calculate AUC
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))

Overwriting src/diabetes-training.py


スクリプトを作成したら、そのスクリプトをジョブとして実行できます。スクリプトは一般的なライブラリを使用する。そのため、pandas、numpy、scikit-learnなどを含むキュレーション環境を使用することができます。

ジョブはキュレーション環境の最新バージョンである`AzureML-sklearn-0.24-ubuntu18.04-py37-cpu`を使用します。

In [5]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-curated-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Monitor your job at https://ml.azure.com/runs/lime_zebra_v4hrbnz2m3?wsid=/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo&tid=adcb56ab-ed63-4be3-8e5e-2fa7d2eb44e1


While the job is running, you can already run the next cells.

## List environments

ワークスペース内の環境を調べてみよう。

前の作業では、キュレーションされた環境の1つを使いました。ワークスペースにすでに存在するすべての環境を探索するには、環境をリストアップします： 

In [6]:
envs = ml_client.environments.list()
for env in envs:
    print(env.name)

DefaultNcdEnv-azureml-automl
AzureML-AI-Studio-Development
AzureML-ACPT-pytorch-1.13-py38-cuda11.7-gpu
AzureML-ACPT-pytorch-1.12-py38-cuda11.6-gpu
AzureML-ACPT-pytorch-1.12-py39-cuda11.6-gpu
AzureML-ACPT-pytorch-1.11-py38-cuda11.5-gpu
AzureML-ACPT-pytorch-1.11-py38-cuda11.3-gpu
AzureML-responsibleai-0.21-ubuntu20.04-py38-cpu
AzureML-responsibleai-0.20-ubuntu20.04-py38-cpu
AzureML-tensorflow-2.5-ubuntu20.04-py38-cuda11-gpu
AzureML-tensorflow-2.6-ubuntu20.04-py38-cuda11-gpu
AzureML-tensorflow-2.7-ubuntu20.04-py38-cuda11-gpu
AzureML-sklearn-1.0-ubuntu20.04-py38-cpu
AzureML-pytorch-1.10-ubuntu18.04-py38-cuda11-gpu
AzureML-pytorch-1.9-ubuntu18.04-py37-cuda11-gpu
AzureML-pytorch-1.8-ubuntu18.04-py37-cuda11-gpu
AzureML-sklearn-0.24-ubuntu18.04-py37-cpu
AzureML-lightgbm-3.2-ubuntu18.04-py37-cpu
AzureML-pytorch-1.7-ubuntu18.04-py37-cuda11-gpu
AzureML-tensorflow-2.4-ubuntu18.04-py37-cuda11-gpu
AzureML-Triton
AzureML-Designer-Score
AzureML-VowpalWabbit-8.8.0
AzureML-PyTorch-1.3-CPU


キュレーションされた環境はすべて **AzureML-** で始まる名前を持っていることに注意してください (この接頭辞はあなた自身の環境には使えません)。

特定の環境を確認するには、その名前とバージョンで環境を検索します。例えば、前のジョブで使用したキュレーション環境の *description* と *tags* を取得できます：

In [7]:
env = ml_client.environments.get("AzureML-sklearn-0.24-ubuntu18.04-py37-cpu", version=44)
print(env. description, env.tags)

An environment for tasks such as regression, clustering, and classification with Scikit-learn. Contains the Azure ML SDK and additional python packages. {'Scikit-learn': '0.24.1', 'OS': 'Ubuntu18.04', 'Training': ''}


## Create and use a custom environment

キュレーションされた環境に、スクリプトの実行に必要なPythonパッケージがすべて含まれていない場合、独自のカスタム環境を作成することができます。環境に必要なパッケージをすべてリストアップすることで、スクリプトを簡単に再実行できます。すべての依存パッケージは環境に保存され、使用するコンピュートとは関係なく、ジョブ構成で指定することができます。

例えば、Dockerイメージから簡単に環境を作成することができます。PyTorchのような特定のフレームワークには、必要なものがすべて含まれているDockerイメージが公開されています。

Dockerイメージから環境を作ってみましょう：

In [8]:
from azure.ai.ml.entities import Environment

env_docker_image = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    name="docker-image-example",
    description="Environment created from a Docker image.",
)
ml_client.environments.create_or_update(env_docker_image)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'docker-image-example', 'description': 'Environment created from a Docker image.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourceGroups/ai-900/providers/Microsoft.MachineLearningServices/workspaces/koikedemo/environments/docker-image-example/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/demokoike/code/Users/wwe.ayumu/azure-ml-labs/Labs/04', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f7861d85720>, 'serialize': <msrest.serialization.Serializer object at 0x7f7861d87d60>, 'version': '1', 'conda_file': None, 'build': None, 'inference_config': No

The environment is now registered in your workspace and you can reference it when you run a script as a job:

In [9]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Monitor your job at https://ml.azure.com/runs/sad_curtain_tdt1gthn51?wsid=/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo&tid=adcb56ab-ed63-4be3-8e5e-2fa7d2eb44e1


<p style=「color:red;font-size:120%;background-color:yellow;font-weight:bold」> ジョブはすぐに失敗します！エラーメッセージを確認してください。</p>
エラーメッセージはpandasという名前のモジュールが存在しないことを示しています。このようなエラーには2つの原因が考えられます：

- スクリプトはpandasを使用しているが、ライブラリをインポートしていない（`import pandas as pd`）。
- スクリプトはpandasを使用しているが、ライブラリがインストールされてない(`pip install pandas`)。

diabetes-training.py`スクリプトを見直すと、スクリプトは正しく、ライブラリがインストールされていないことがわかります。つまり、必要なパッケージが環境に含まれていなかったのです。

前のジョブで使ったベースDockerイメージを使って、新しい環境を作りましょう。ここで、必要なパッケージがインストールされるようにcondaの仕様を追加する。まず、以下のセルを実行してcondaの仕様ファイルを作成します：


In [10]:
%%writefile src/conda-env.yml
name: basic-env-cpu
channels:
  - conda-forge
dependencies:
  - python=3.11
  - scikit-learn
  - pandas
  - numpy
  - matplotlib

Writing src/conda-env.yml


スクリプトを正常に実行するために必要な依存関係はすべてconda仕様ファイルに含まれていることに注意してください。

ベースとなる Docker イメージと conda 仕様ファイルを使用して新しい環境を作成し、必要な依存関係を追加します。Azure Machine Learning は、提供された Docker イメージの上に conda 環境を構築します。

In [11]:
from azure.ai.ml.entities import Environment

env_docker_conda = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src/conda-env.yml",
    name="docker-image-plus-conda-example",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env_docker_conda)

Environment({'arm_type': 'environment_version', 'latest_version': None, 'image': 'mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04', 'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'docker-image-plus-conda-example', 'description': 'Environment created from a Docker image plus Conda environment.', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': False, 'id': '/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourceGroups/ai-900/providers/Microsoft.MachineLearningServices/workspaces/koikedemo/environments/docker-image-plus-conda-example/versions/1', 'Resource__source_path': '', 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/demokoike/code/Users/wwe.ayumu/azure-ml-labs/Labs/04', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f7869158cd0>, 'serialize': <msrest.serialization.Serializer object at 0x7f7861ddbd90>, 'version': '1', 'conda_file'

Now, you can submit a job with the new environment to run the script:

In [12]:
from azure.ai.ml import command

# configure job
job = command(
    code="./src",
    command="python diabetes-training.py",
    environment="docker-image-plus-conda-example:1",
    compute="aml-cluster",
    display_name="diabetes-train-custom-env",
    experiment_name="diabetes-training"
)

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Uploading src (0.52 MBs): 100%|██████████| 519350/519350 [00:00<00:00, 10576536.76it/s]




Monitor your job at https://ml.azure.com/runs/sweet_parcel_2n9fgq4p54?wsid=/subscriptions/2fbd9cca-a295-4d4f-a5c4-bf80e223260f/resourcegroups/ai-900/workspaces/koikedemo&tid=adcb56ab-ed63-4be3-8e5e-2fa7d2eb44e1


新しいカスタム環境でジョブを投入すると、環境の構築が開始されます。新しく作成した環境を初めて使用する場合、環境の構築に 10-15 分かかることがあります。

ジョブを投入する前に手動で環境構築をトリガーすることもできます。環境を構築する必要があるのは、最初に使用するときだけです。

ジョブが新しい環境のビルドをトリガーすると、ジョブの **Outputs + logs** タブでビルドのログを確認できます。**azureml-logs/20_image_build_log.txt**を開いて、環境構築のログを確認してください。

スクリーンショットのビルドログ](./images/screenshot-logs.png)